In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver

In [ ]:
def NaverWebtoon(end_page=1):
    base_url = 'https://comic.naver.com/webtoon/detail.nhn?titleId=626907&no={}&weekday=wed'
    driver = webdriver.Chrome()
    comment_url = 'https://comic.naver.com/comment/comment.nhn?titleId=626907&no={}#'
    # 날짜, 평점
    result = []
    for page in range(1, end_page+1):
        # url 불러오기
        url = base_url.format(page)
        c_url = comment_url.format(page)
        res = requests.get(url)
        driver.get(c_url)
        c_box = driver.find_elements_by_css_selector('div.u_cbox_comment_box')
        c_list = []
        for i in range(c_box.__len__()):
            try:
                c_list.append(c_box[i].find_element_by_css_selector('div.u_cbox_comment_box span.u_cbox_contents').text)
            except:
                c_list.append(c_box[i].find_element_by_css_selector('div.u_cbox_comment_box span.u_cbox_cleanbot_contents').text)
#           print(cList)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, 'lxml')
            title = soup.select_one('div.view h3').get_text().strip()
            point_list = soup.select_one('span#topPointTotalNumber').get_text().strip()
            re_date = soup.select_one('dd.date').get_text().strip()
            result.append([re_date, title, point_list, c_list])
#           print(title, re_date, point_list, c_list)
    print(result)
#   print(result + cList)
    filename = "webtoon_{}.csv".format(end_page) # 첫번째 줄에 랭크 카테고리, 두번째 줄에 날짜
    df = pd.DataFrame(result, columns =['등록일', '제목', '평점', '베댓'])
    # print(df)
    df.to_csv(filename, index=False, encoding = 'UTF-8')
    return filename
if __name__=='__main__':
    NaverWebtoon(end_page=276)